In [4]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [5]:
pip install --upgrade pip


     ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
      --------------------------------------- 0.0/1.8 MB 960.0 kB/s eta 0:00:02
     - -------------------------------------- 0.1/1.8 MB 777.7 kB/s eta 0:00:03
     --- ------------------------------------ 0.1/1.8 MB 1.1 MB/s eta 0:00:02
     --- ------------------------------------ 0.2/1.8 MB 913.1 kB/s eta 0:00:02
     ---- ----------------------------------- 0.2/1.8 MB 903.5 kB/s eta 0:00:02
     ---- ----------------------------------- 0.2/1.8 MB 903.5 kB/s eta 0:00:02
     ---- ----------------------------------- 0.2/1.8 MB 903.5 kB/s eta 0:00:02
     ----- ---------------------------------- 0.2/1.8 MB 625.8 kB/s eta 0:00:03
     ------ --------------------------------- 0.3/1.8 MB 710.0 kB/s eta 0:00:03
     ------ --------------------------------- 0.3/1.8 MB 654.1 kB/s eta 0:00:03
     ------ --------------------------------- 0.3/1.8 MB 633.4 kB/s eta 0:00:03
     -------- ------------------------------- 0.4/1


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: C:\Users\tejag\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
%pip install --upgrade pandas
%pip install --upgrade numpy


   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.3/11.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.6 MB 1.2 MB/s eta 0:00:10
   -- ------------------------------------- 0.8/11.6 MB 1.1 MB/s eta 0:00:11
   --- ------------------------------------ 1.0/11.6 MB 1.1 MB/s eta 0:00:10
   ---- ----------------------------------- 1.3/11.6 MB 1.2 MB/s eta 0:00:09
   ------ --------------------------------- 1.8/11.6 MB 1.3 MB/s eta 0:00:08
   -------- ------------------------------- 2.4/11.6 MB 1.5 MB/s eta 0:00:07
   --------- ------------------------------ 2.6/11.6 MB 1.5 MB/s eta 0:00:06
   ----------- ---------------------------- 3.4/11.6 MB 1.7 MB/s eta 0:00:05
   ------------- -------------------------- 3.9/11.6 MB 1.8 MB/s eta 0:00:05
   --------------- ------------------------ 4.5/11.6 MB 1.9 MB/s eta 0:00:04
   ----------------